In [5]:
import torch
import numpy as np

from lab9.Understanding_PyTorch_Copy import joint_dataset

In [6]:
torch.manual_seed(0)
t1 = 2 * torch.rand(5,2) - 1
t2 = torch.normal(mean=0, std=1, size=(5,2))
print(t1)
print(t2)

tensor([[-0.0075,  0.5364],
        [-0.8230, -0.7359],
        [-0.3852,  0.2682],
        [-0.0198,  0.7929],
        [-0.0887,  0.2646]])
tensor([[ 0.1604, -0.6065],
        [-0.7831,  1.0622],
        [-0.2613,  1.0667],
        [ 0.4159,  0.8396],
        [-0.8265, -0.7949]])


In [11]:
# Perform element-wise multiplication of matrix
torch.multiply(t1, t2)

tensor([[-0.0012, -0.3253],
        [ 0.6445, -0.7817],
        [ 0.1006,  0.2860],
        [-0.0082,  0.6657],
        [ 0.0733, -0.2104]])

In [16]:
# Perform dot product
torch.mm(t1, t2.T)

tensor([[-0.3265,  0.5757,  0.5742,  0.4473, -0.4203],
        [ 0.3143, -0.1372, -0.5700, -0.9602,  1.2652],
        [-0.2244,  0.5864,  0.3867,  0.0650,  0.1051],
        [-0.4840,  0.8577,  0.8510,  0.6574, -0.6139],
        [-0.1747,  0.3506,  0.3055,  0.1853, -0.1370]])

## Split, stack and concatenate tensors

In [23]:
torch.manual_seed(0)
t = torch.randn(6)
print(f'Tensor t : {t}')
t_splits = torch.chunk(t, 3)
[item.numpy() for item in t_splits]

Tensor t : tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845, -1.3986])


[array([ 1.5409961, -0.2934289], dtype=float32),
 array([-2.1787894 ,  0.56843126], dtype=float32),
 array([-1.0845224, -1.3985955], dtype=float32)]

In [24]:
torch.manual_seed(0)
t = torch.rand(8)
print(f'Tensor t : {t}')
t_splits = torch.chunk(t, 3) # 8 ! // 3 so the last element will not have the same number as others
[item.numpy() for item in t_splits]

Tensor t : tensor([0.4963, 0.7682, 0.0885, 0.1320, 0.3074, 0.6341, 0.4901, 0.8964])


[array([0.4962566 , 0.7682218 , 0.08847743], dtype=float32),
 array([0.13203049, 0.30742282, 0.6340787 ], dtype=float32),
 array([0.4900934 , 0.89644474], dtype=float32)]

## Stacking

In [30]:
A = torch.ones(3)
B = torch.zeros(2)

C = torch.cat([A, B], axis=0)
C

tensor([1., 1., 1., 0., 0.])

In [32]:
A = torch.ones(3)
B = torch.zeros(3)

C = torch.stack([A, B], axis=1) # Column stacking
C

tensor([[1., 0.],
        [1., 0.],
        [1., 0.]])

## DataLoader

In [33]:
from torch.utils.data import DataLoader

t = torch.arange(6, dtype=torch.float32)
data_loader = DataLoader(t)

In [34]:
type(data_loader) # DataLoader is an iterator

torch.utils.data.dataloader.DataLoader

In [35]:
for item in data_loader:
    print(item)

tensor([0.])
tensor([1.])
tensor([2.])
tensor([3.])
tensor([4.])
tensor([5.])


Now creating batch from dataset with desired batch size e.g 3

In [36]:
data_loader = DataLoader(t, batch_size=3, drop_last=False)
for i, batch in enumerate(data_loader):
    print(f'Batch {i} : {batch}')

Batch 0 : tensor([0., 1., 2.])
Batch 1 : tensor([3., 4., 5.])


Combining two tensors into a joint dataset

In [37]:
torch.manual_seed(0)
t_x = torch.rand([4,3], dtype=torch.float32)
t_y = torch.arange(4)

In [43]:
from torch.utils.data import Dataset

class JointDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [45]:
joint_dataset = JointDataset(t_x, t_y)
print(type(joint_dataset))
for example in joint_dataset:
    print(f' x : {example[0]} , y : {example[1]}')

<class '__main__.JointDataset'>
 x : tensor([0.4963, 0.7682, 0.0885]) , y : 0
 x : tensor([0.1320, 0.3074, 0.6341]) , y : 1
 x : tensor([0.4901, 0.8964, 0.4556]) , y : 2
 x : tensor([0.6323, 0.3489, 0.4017]) , y : 3


Shuffle Batch and repeat

In [47]:
torch.manual_seed(0)
data_loader = DataLoader(joint_dataset, batch_size=2, shuffle=True)
for i, batch in enumerate(data_loader):
    print(f'Batch {i} : X : {batch[0]} , Y : {batch[1]}')

Batch 0 : X : tensor([[0.4901, 0.8964, 0.4556],
        [0.1320, 0.3074, 0.6341]]) , Y : tensor([2, 1])
Batch 1 : X : tensor([[0.6323, 0.3489, 0.4017],
        [0.4963, 0.7682, 0.0885]]) , Y : tensor([3, 0])


In [48]:
for epoch in range(2):
    print(f'Epoch {epoch}')
    for i, batch in enumerate(data_loader):
        print(f' x : {batch[0]} , y : {batch[1]}')

Epoch 0
 x : tensor([[0.4963, 0.7682, 0.0885],
        [0.1320, 0.3074, 0.6341]]) , y : tensor([0, 1])
 x : tensor([[0.6323, 0.3489, 0.4017],
        [0.4901, 0.8964, 0.4556]]) , y : tensor([3, 2])
Epoch 1
 x : tensor([[0.1320, 0.3074, 0.6341],
        [0.4963, 0.7682, 0.0885]]) , y : tensor([1, 0])
 x : tensor([[0.6323, 0.3489, 0.4017],
        [0.4901, 0.8964, 0.4556]]) , y : tensor([3, 2])
